In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import tensorflow as tf

from thesis import pqmf
from ddsp.colab.colab_utils import (
    auto_tune, get_tuning_factor, download,
    play, record, specplot, upload, audio_bytes_to_np,
    DEFAULT_SAMPLE_RATE)
import numpy as np
import IPython

def play_audio(audio):
    audio = np.array(audio)
    audio = np.squeeze(audio)
    IPython.display.display(IPython.display.Audio(audio, rate=16000))

from ddsp.losses import SpectralLoss

def get_loss(target_audio, audio):
    loss_class = SpectralLoss()
    loss = loss_class(tf.convert_to_tensor(target_audio), tf.convert_to_tensor(audio))
    return float(loss)

In [ ]:
sample_rate = 16000
bands = 8
attenuation = 100

input_f = open("../data/audio/violin/II. Double.mp3", "rb")
wav_bytes = input_f.read()
audio = audio_bytes_to_np(wav_bytes)
audio = audio[:sample_rate * 10]
audio_np = audio.copy()

if len(audio.shape) == 1:
    audio = audio[np.newaxis, :, np.newaxis]

audio_tf = tf.convert_to_tensor(audio)

In [ ]:
play_audio(audio[0,:,0])

In [ ]:
p = pqmf.PQMF(pqmf.MultiBandMelGANGeneratorConfig(subbands=8, beta=9, cutoff_ratio=0.05))

In [ ]:
p.synthesis_filter.shape

In [ ]:
%%timeit -n100
# Analysis
a = p.analysis(audio_tf)
# print("Analyzed to", a.shape)

rec_tf = p.synthesis(a)
# print("Synthesized to", rec_tf.shape)

# play_audio(rec_tf[0,:,0])
# get_loss(audio_tf[:,:,0], rec_tf[:,:,0])

In [ ]:
# Upsample to hear what it sounds like

from thesis.newt import resample

a2 = resample(a, a.shape[1] * 8)

a2.shape

for i in range(8):
    IPython.display.display(IPython.display.Audio(0.5 * a2[:,:sample_rate*10,i], rate=sample_rate))

In [ ]:
specplot(audio[0])

In [ ]:
# Show spectrograms to visualize
for i in range(8):
    specplot(a2[:,:,i])

In [ ]:
get_loss(audio_torch[:,:,0], rec_torch[0])

In [ ]:
IPython.display.display(IPython.display.Audio(reconstruction[0,:,0], rate=sample_rate))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(p.h)

In [ ]:
for i in range(8):
    plt.plot(p.hk[i][64:-64])
    plt.show()

In [ ]:
import torch
from thesis import pqmf0

audio_torch = torch.Tensor(audio_np).reshape((1, 1, -1))

p0 = pqmf0.PQMF(attenuation=attenuation, n_band=bands)

In [ ]:
# %%timeit
temp = p0.forward(audio_torch)
print("Analyzed to", temp.shape)
rec_torch = p0.inverse(temp)
# print("Synthesized to", rec_torch.shape)

In [ ]:
# Upsample to hear what it sounds like

from thesis.newt import resample

a2 = resample(temp.swapaxes(1, 2), temp.shape[2] * 8)

print(a2.shape)

for i in range(8):
    play_audio(a2[:,:sample_rate*3, i])

In [ ]:
temp0 = p0.forward(audio_torch)
print("Analyzed to", temp0.shape)
rec_torch = p0.inverse(temp0)
print("Synthesized to", rec_torch.shape)

play_audio(rec_torch)

get_loss(audio_torch[0], rec_torch[0])

In [ ]:
p0.hk.shape

## Custom PQMF

In [ ]:
from thesis import pqmf1

# audio_torch = torch.Tensor(audio_np).reshape((1, 1, -1))
p1 = pqmf1.PQMF(attenuation=attenuation, n_band=bands, polyphase=False)

audio_tf1 = tf.convert_to_tensor(audio_np)
audio_tf1 = audio_tf1.reshape((1, -1, 1))

In [ ]:
temp = p1.analysis(audio_tf1)

In [ ]:
temp.shape

In [ ]:
# Upsample to hear what it sounds like

from thesis.newt import resample

a2 = resample(temp, temp.shape[1] * 8)

print(a2.shape)

for i in range(8):
    play_audio(a2[:,:sample_rate*8,i])

In [ ]:
p1.synthesis(temp)